# Mix

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt

from scipy import ndimage 

import os
import cv2
import pandas as pd
%matplotlib inline

In [2]:
#Load the coordinates data set
df = pd.read_csv('train.csv')

In [3]:
# Load images and resize 
root= './train'
names = []
train_crude = []
for file in os.listdir(root):
    img = cv2.imread(os.path.join(root, file))#,cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img.shape[1]//3,img.shape[0]//3))
    train_crude.append(img)
    names.append(file.split('.')[0])
    
data = np.stack( train_crude, axis=0 )

### Extract feature CNN

In [4]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
base_model = ResNet50V2(weights='imagenet',input_shape=[163, 226,3],include_top=False)

In [5]:
#get features in batches and flatten
feat = []
for i in range(0,7500,500):
    temp = base_model.predict(preprocess_input(data[i:i+500]))
    feat.append(temp)
    

feat0 = np.concatenate(feat, axis=0) 
feat2 = [i.ravel() for i in feat0]
feat2 = np.stack(feat2, axis=0)

### Nearest neighbor

In [7]:

from sklearn.neighbors import NearestNeighbors
train = feat2[:7000]
validation = feat2[7000:]

neighTest = NearestNeighbors(n_neighbors=5)
neighTest.fit(train)

predict = neighTest.kneighbors(validation)

In [9]:
predict[0][0]

array([437.6941 , 452.0017 , 455.92123, 459.71375, 459.81705],
      dtype=float32)

In [10]:
predict[1][0]

array([2964,  684,  322, 3702, 6431], dtype=int64)

### SIFT